In [46]:
import pandas as pd
import seaborn as sns
import re
import numpy as np

In [47]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [57]:
def get_title(name):
    match = re.search(' ([A-Za-z]+)\.', name)
    if match:
        return match.group()
    else:
        return NaN

In [ ]:
def get_group(age):
    if(age<10):
        return 1
    elif (age<20):
        return 2
    elif (age<30):
        return 3
    elif (age<40):
        return 4
    elif (age<50):
        return 5
    elif (age<60):
        return 6
    elif (age<70):
        return 7
    elif(age<80):
        return 8
    else:
        return 10

In [49]:
sex_map = {"male":0,"female":1}

embarked_map = {'S':0,'Q':1,'C':2}

boolean_map = {False:0,True:1}

title_map = {' Mr.':0.1, ' Mrs.':0.2, ' Miss.':0.3, ' Master.':0.4, ' Don.':0.5, ' Rev.':0.6, ' Dr.':0.7, ' Mme.':0.8, ' Ms.':0.9,
 ' Major.':1.1, ' Lady.':1.2, ' Sir.':1.3, ' Mlle.':1.4, ' Col.':1.5, ' Capt.':1.6, ' Countess.':1.7,' Jonkheer.':1.8}#해당 수치 조절 필요할듯.. Title이 가지는 의미에 대해 같이 고민해봅시당 

train['Sex'] = train['Sex'].map(sex_map)

train['Embarked'] = train['Embarked'].map(embarked_map)
train['Embarked'] = train['Embarked'].fillna(0)#누락된값이 2개밖에 없어서 그냥 S로 했습니다.

train['Cabin'] = train['Cabin'].notnull()
train['Cabin'] = train['Cabin'].map(boolean_map)

train['Title'] = train['Name'].map(get_title)
train['Title'] = train['Title'].map(title_map)
train['Title'] = train['Title'].fillna(1)

#train['Age'] = train[['Age','Title']].apply(guess_age,axis=1)#비어있는 Age에 대해 각 title별 평균나이 적용 했어요

#train[['Title','Age']].groupby(['Title']).mean() title별 Age의 평균
test['Sex'] = test['Sex'].map(sex_map)

test['Embarked'] = test['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].fillna(0)

test['Cabin'] = test['Cabin'].notnull()
test['Cabin'] = test['Cabin'].map(boolean_map)#Cabin의 경우에 기록 유무에만 따라 0 1 로 했어요

test['Title'] = test['Name'].map(get_title)
test['Title'] = test['Title'].map(title_map)
test['Title'] = test['Title'].fillna(3)#이것두 Title의 조정 수치

test['Agegroup'] = test['Age'].map(get_group)
#test['Age'] = test[['Age','Title']].apply(guess_age,axis=1)

train.drop('PassengerId',axis=1,inplace=True)
train.drop('Name',axis=1,inplace=True)
train.drop('Ticket',axis=1,inplace=True)
train.drop('Survived',axis=1,inplace=True)

test.drop('Name',axis=1,inplace=True)
test.drop('Ticket',axis=1,inplace=True)

test_data=test.drop('PassengerId',axis=1).copy()

In [56]:
nulled_age_data=train[train['Age'].isnull()]
notnulled_age_data=train[train['Age'].notnull()]
print(notnulled_age_data)

     Pclass  Sex   Age  SibSp  Parch      Fare  Cabin  Embarked  Title
0         3    0  22.0      1      0    7.2500      0       0.0    0.1
1         1    1  38.0      1      0   71.2833      1       2.0    0.2
2         3    1  26.0      0      0    7.9250      0       0.0    0.3
3         1    1  35.0      1      0   53.1000      1       0.0    0.2
4         3    0  35.0      0      0    8.0500      0       0.0    0.1
6         1    0  54.0      0      0   51.8625      1       0.0    0.1
7         3    0   2.0      3      1   21.0750      0       0.0    0.4
8         3    1  27.0      0      2   11.1333      0       0.0    0.2
9         2    1  14.0      1      0   30.0708      0       2.0    0.2
10        3    1   4.0      1      1   16.7000      1       0.0    0.3
11        1    1  58.0      0      0   26.5500      1       0.0    0.3
12        3    0  20.0      0      0    8.0500      0       0.0    0.1
13        3    0  39.0      1      5   31.2750      0       0.0    0.1
14    

In [53]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tensorflow.contrib import learn

y_data = notnulled_age_data['Age']
x_data = notnulled_age_data[['Pclass','Sex','SibSp','Parch','Fare','Cabin','Embarked','Title']]
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,random_state=42)#교차검증 데이터셋
#x_train = x_data
#y_train = y_data

In [54]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
classifier = learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10,20,10], n_classes=2)
classifier.fit(x_train, y_train,steps=3000) #hidden layer 3개의 초기 모델

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B0A841E400>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ljh_9\\AppData\\Local\\Temp\\tmp8bo14hws'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving check

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x000001B0A8673748>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=8, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x000001B0A1DA57B8>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [55]:
predicted = classifier.predict(x_test)
print(accuracy_score(y_test,list(predicted)))
#교차검증 확인

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ljh_9\AppData\Local\Temp\tmp8bo14hws\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


ValueError: Classification metrics can't handle a mix of continuous and binary targets